In [1]:
# Install necessary libraries if not already installed
!pip install -q transformers datasets tqdm

# Import necessary libraries
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from datasets import load_dataset
import torch
import re
from tqdm import tqdm
import random

# Check if CUDA (GPU) is available, and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the GPT-Neo model and tokenizer, and move the model to the device
model_name = "EleutherAI/gpt-neo-2.7B"
model = GPTNeoForCausalLM.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load the MedQA dataset with training and testing splits
dataset = load_dataset("GBaker/MedQA-USMLE-4-options")
train_dataset = dataset['train']
test_dataset = dataset['test']

Using device: cuda


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [22]:


# Function to perform inference with few-shot prompting and match answer using regex
def get_model_prediction(question, options_dict):
    options_text = ', '.join([f"{key}: {value}" for key, value in options_dict.items()])
    input_text = f"Question: {question}\nOptions: {options_text}\nThink like a medical professional step by step.\nAnswer:"

#     print(input_text)
#     print("*****************************")
    # Tokenize input text
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate output (can be tuned using temperature, max_length, etc.)
    output = model.generate(inputs["input_ids"], max_new_tokens=500)

    # Decode the output to text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
#     print(generated_text)
    # Extract the answer after "Answer:" using regex
    match = re.search(r"Answer:\s*([A-D])", generated_text)
    
    if match:
        predicted_option = match.group(1)
    else:
        predicted_option = None  # Handle case where no match is found
    
    return predicted_option


In [23]:

# Evaluate model on the test dataset with tqdm to monitor the progress
correct_predictions = 0
total_predictions = 0

# Add tqdm to track the evaluation loop
for sample in tqdm(test_dataset, desc="Evaluating on test dataset", total=len(test_dataset)):
    question = sample['question']
    options = sample['options']  # This is a dictionary with 'A', 'B', 'C', 'D' as keys
    correct_answer = sample['answer_idx']  # Correct answer is the key ('A', 'B', 'C', or 'D')

    # Get the model's prediction using few-shot prompting
    predicted_answer = get_model_prediction(question, options)
#     print(predicted_answer)
#     print(correct_answer)
    # Check if prediction is correct
    if predicted_answer == correct_answer:
        correct_predictions += 1
    
    total_predictions += 1

    # Calculate and print the accuracy after each iteration
    accuracy = correct_predictions / total_predictions * 100
    print(f"Iteration {total_predictions}: Accuracy so far: {accuracy:.2f}%")
    

# Final accuracy
final_accuracy = correct_predictions / total_predictions * 100
print(f"Final Accuracy: {final_accuracy:.2f}%")

Evaluating on test dataset:   0%|          | 0/1273 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Evaluating on test dataset:   0%|          | 1/1273 [00:28<10:09:12, 28.74s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Iteration 1: Accuracy so far: 100.00%


Evaluating on test dataset:   0%|          | 2/1273 [00:57<10:07:20, 28.67s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Iteration 2: Accuracy so far: 50.00%


Evaluating on test dataset:   0%|          | 3/1273 [01:26<10:14:51, 29.05s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Iteration 670: Accuracy so far: 24.93%


Evaluating on test dataset:  53%|█████▎    | 671/1273 [5:24:04<4:47:57, 28.70s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Iteration 671: Accuracy so far: 24.89%


Evaluating on test dataset:  53%|█████▎    | 671/1273 [5:24:08<4:50:48, 28.98s/it]


KeyboardInterrupt: 